# How to add ad-hoc tool calling capability to LLMs and Chat Models

To add ad-hoc tool calling capability to LLMs and chat models, we can use LangChain's tool system and create a prompt that instructs the model to invoke the right tool based on the user's input. Here's a step-by-step breakdown of the process:

In [1]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")


C:\Users\Admin\AppData\Local\Temp\ipykernel_8524\2418719106.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model="phi3")


In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")


# Create a Tool
You can define custom tools using the @tool decorator. Here's an example of two simple tools: multiply and add:

In [3]:
from langchain_core.tools import tool

@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y

@tool
def add(x: int, y: int) -> int:
    """Add two numbers together."""
    return x + y

tools = [multiply, add]


# Create the Prompt
We need a prompt that specifies the tools available, their descriptions, and the output format (JSON):

In [4]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

# Render tool descriptions
rendered_tools = render_text_description(tools)

system_prompt = f"""
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

# Create prompt template
prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("user", "{input}")])


# Chain Setup
Next, create a chain that invokes the model and parses the output as JSON:

In [5]:
chain = prompt | model | JsonOutputParser()


# Invoke the Tool
Now, the model will output which tool to use, along with the required arguments. We'll implement the logic to actually invoke the selected tool.

In [6]:
from typing import Any, Dict, Optional, TypedDict

class ToolCallRequest(TypedDict):
    """A typed dict for tool invocation inputs."""
    name: str
    arguments: Dict[str, Any]

def invoke_tool(tool_call_request: ToolCallRequest):
    """Invoke the specified tool."""
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"])

# Example invocation
result = invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})
print(result)  # Output: 15.0


15.0


# Combine Everything
Finally, chain everything together and invoke the chain:

In [7]:
chain = prompt | model | JsonOutputParser() | invoke_tool
response = chain.invoke({"input": "what's thirteen times 4"})
print(response)


52.0
